In [1]:
# Sistema e utilitários
import os
import gc
from copy import deepcopy
from datetime import datetime

# Bibliotecas principais
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Scikit-learn
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, QuantileTransformer
from sklearn.datasets import load_iris

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# Keras4Torch
import keras4torch
from keras4torch.callbacks import ModelCheckpoint, LRScheduler

# STab
from stab import MainModel
from stab.model_wrapper import Num_Cat

In [2]:
# Carregar Iris
iris = load_iris()
X = iris.data  # Apenas variáveis numéricas
y = iris.target

# Dividir em treino, validação e teste
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Normalizar
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train).astype(np.float32)
X_valid = scaler.transform(X_valid).astype(np.float32)
X_test  = scaler.transform(X_test).astype(np.float32)

# Ajustar rótulos para PyTorch
y_train = torch.tensor(y_train, dtype=torch.long)
y_valid = torch.tensor(y_valid, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

In [3]:
# Definir o modelo (sem categorias)
Or_model = MainModel(
    categories        = (),   # Sem features categóricas
    num_continuous    = 4,
    dim               = 16,
    dim_out           = 3,    # 3 classes (Iris)
    depth             = 2,
    heads             = 4,
    attn_dropout      = 0.1,
    ff_dropout        = 0.1,
    U                 = 2,
    cases             = 8,
)


In [4]:
# Criar o wrapper Num_Cat para gerenciar as entradas corretamente
no_model = Num_Cat(Or_model, num_number=4, classes=3, Sample_size=16)  # 4 features numéricas, 3 classes (Iris)


In [5]:
# Passar o modelo para keras4torch
model = keras4torch.Model(no_model).build([4])  # Agora só precisa da dimensão numérica


optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.0001)
model.compile(optimizer=optimizer, loss=F.cross_entropy, metrics=['accuracy'])

In [6]:
# Treinamento (passando apenas X numérico)
model.fit([X_train], y_train,  # Ajuste aqui: lista com X_train
          epochs=15, batch_size=16,
          validation_data=([X_valid], y_valid),  # Ajuste aqui
          verbose=2)

Train on 105 samples, validate on 22 samples:
Epoch 1/15 - 2.4s - loss: 1.0076 - acc: 0.4762 - val_loss: 0.8069 - val_acc: 0.9091 - lr: 1e-03
Epoch 2/15 - 1.7s - loss: 0.8170 - acc: 0.6286 - val_loss: 0.6136 - val_acc: 0.9091 - lr: 1e-03
Epoch 3/15 - 1.7s - loss: 0.7219 - acc: 0.6381 - val_loss: 0.4963 - val_acc: 0.9545 - lr: 1e-03
Epoch 4/15 - 1.7s - loss: 0.5785 - acc: 0.7619 - val_loss: 0.4412 - val_acc: 0.8182 - lr: 1e-03
Epoch 5/15 - 1.7s - loss: 0.5059 - acc: 0.7905 - val_loss: 0.3596 - val_acc: 0.9091 - lr: 1e-03
Epoch 6/15 - 1.7s - loss: 0.5151 - acc: 0.7905 - val_loss: 0.3478 - val_acc: 0.9545 - lr: 1e-03
Epoch 7/15 - 1.8s - loss: 0.4192 - acc: 0.8476 - val_loss: 0.3154 - val_acc: 0.9091 - lr: 1e-03
Epoch 8/15 - 1.7s - loss: 0.3844 - acc: 0.8762 - val_loss: 0.2691 - val_acc: 0.9545 - lr: 1e-03
Epoch 9/15 - 1.7s - loss: 0.4096 - acc: 0.8667 - val_loss: 0.2492 - val_acc: 0.9545 - lr: 1e-03
Epoch 10/15 - 1.7s - loss: 0.3808 - acc: 0.8762 - val_loss: 0.2456 - val_acc: 0.9545 - lr:

,loss,acc,val_loss,val_acc,lr
1,1.007578,0.476190,0.806882,0.909091,0.001
2,0.816983,0.628571,0.613628,0.909091,0.001
3,0.721907,0.638095,0.496290,0.954545,0.001
4,0.578453,0.761905,0.441236,0.818182,0.001
5,0.505929,0.790476,0.359630,0.909091,0.001
6,0.515063,0.790476,0.347758,0.954545,0.001
7,0.419229,0.847619,0.315401,0.909091,0.001
8,0.384427,0.876190,0.269130,0.954545,0.001
9,0.409555,0.866667,0.249158,0.954545,0.001
10,0.380806,0.876190,0.245588,0.954545,0.001


In [7]:
# Avaliação
test_logits = model.predict([X_test])  # Passar X_test como lista
test_preds = np.argmax(test_logits, axis=1)
accuracy = np.mean(test_preds == y_test.numpy())
print(f'Test Accuracy: {accuracy:.4f}')

report = classification_report(y_test.numpy(), test_preds, target_names=iris.target_names)
print(report)


Test Accuracy: 0.8696
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00         6
  versicolor       1.00      0.70      0.82        10
   virginica       0.70      1.00      0.82         7

    accuracy                           0.87        23
   macro avg       0.90      0.90      0.88        23
weighted avg       0.91      0.87      0.87        23

